# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key
from pprint import pprint

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,palmer,42.1584,-72.3287,14.66,82,75,0.00,US,1683521439
1,1,ribeira grande,38.5167,-28.7000,16.54,96,100,4.02,PT,1683521439
2,2,ulsteinvik,62.3432,5.8487,11.28,52,32,1.34,NO,1683521439
3,3,vilyuysk,63.7553,121.6247,-3.99,67,98,2.98,RU,1683521439
4,4,la passe,45.5549,-0.8967,13.88,91,48,0.43,FR,1683521439


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    cmap="viridis",
    colorbar=True,
    title="City Humidity",
    height=500,
    width=800,
    size=city_data_df["Humidity"]*10,
    alpha=0.5,
    hover_cols=["City", "Country", "Humidity"],
    geo = True,
    tiles="ESRI"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 30) & 
                           (city_data_df["Wind Speed"] < 10)&
                           (city_data_df["Humidity"]>55)& (city_data_df["Humidity"]<65)]


# Drop any rows with null values
criteria_df = criteria_df.dropna()

# Display sample data
criteria_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,dhuusamarreeb,5.5350,46.3861,27.52,60,53,8.71,SO,1683521440
74,74,adamstown,-25.0660,-130.1015,21.88,59,100,9.85,PN,1683521448
86,86,mukalla,14.5425,49.1242,29.08,59,11,1.96,YE,1683521449
87,87,bardai,11.6500,7.2333,23.31,61,89,3.72,NG,1683521450
197,197,cabo san lucas,22.8909,-109.9124,26.95,57,97,6.26,MX,1683521464


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_df[["City","Country","Lat","Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,dhuusamarreeb,SO,5.5350,46.3861,60,
74,adamstown,PN,-25.0660,-130.1015,59,
86,mukalla,YE,14.5425,49.1242,59,
87,bardai,NG,11.6500,7.2333,61,
197,cabo san lucas,MX,22.8909,-109.9124,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 1000
limit = 20

params = {
    "radius" : radius,
    "apiKey":geoapify_key,
    "limit":limit,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"]=f"accommodation.hotel"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dhuusamarreeb - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
mukalla - nearest hotel: فندق الجابري
bardai - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
ankazoabo - nearest hotel: No hotel found
aweil - nearest hotel: Aweil Grand Hotel
elumalai - nearest hotel: No hotel found
wenzhou - nearest hotel: 王朝大酒店
harur - nearest hotel: No hotel found
pamekasan - nearest hotel: No hotel found
samalut - nearest hotel: No hotel found
am timan - nearest hotel: No hotel found
jingdezhen - nearest hotel: 文苑商务大酒店


,City,Country,Lat,Lng,Humidity,Hotel Name
8,dhuusamarreeb,SO,5.5350,46.3861,60,No hotel found
74,adamstown,PN,-25.0660,-130.1015,59,No hotel found
86,mukalla,YE,14.5425,49.1242,59,فندق الجابري
87,bardai,NG,11.6500,7.2333,61,No hotel found
197,cabo san lucas,MX,22.8909,-109.9124,57,Comfort Rooms
215,ankazoabo,MG,-22.2833,44.5167,57,No hotel found
310,aweil,SS,8.7682,27.4002,58,Aweil Grand Hotel
393,elumalai,IN,9.8667,77.7000,61,No hotel found
400,wenzhou,CN,27.9994,120.6668,64,王朝大酒店
445,harur,IN,12.0667,78.5000,62,No hotel found


In [24]:
name_address = response.json()
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': '文苑商务大酒店',
    'country': 'China',
    'country_code': 'cn',
    'region': 'Jingdezhen',
    'state': 'Jiangxi',
    'city': 'Zhushan District',
    'town': 'Zhoulukou Subdistrict',
    'street': '通站路',
    'lon': 117.216014,
    'lat': 29.2897275,
    'formatted': '文苑商务大酒店, China, Jiangxi, Zhushan District, 通站路',
    'address_line1': '文苑商务大酒店',
    'address_line2': 'China, Jiangxi, Zhushan District, 通站路',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': '文苑商务大酒店',
      'osm_id': 4907722103,
      'tourism': 'hotel',
      'osm_type': 'n'}},
    'distance': 962,
    'place_id': '519a5e622cd34d5d4059f16cd8942b4a3d40f00103f90177e5852401000000920315e69687

In [31]:
print(response.url)

'https://api.geoapify.com/v2/places?radius=1000&apiKey=e8c8a92cf3b6413d9544611d5d5132a2&limit=20&filter=circle%3A117.2079%2C29.2947%2C1000&bias=proximity%3A117.2079%2C29.2947&categories=accommodation.hotel'


In [25]:
name_address.keys()

dict_keys(['type', 'features'])

In [27]:
pprint(name_address)

{'features': [{'geometry': {'coordinates': [117.216014, 29.289727499807615],
                            'type': 'Point'},
               'properties': {'address_line1': '文苑商务大酒店',
                              'address_line2': 'China, Jiangxi, Zhushan '
                                               'District, 通站路',
                              'categories': ['accommodation',
                                             'accommodation.hotel'],
                              'city': 'Zhushan District',
                              'country': 'China',
                              'country_code': 'cn',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database Licence',
                                             'raw': {'name': '文苑商务大酒店',
                                                     'osm_id': 4907722103,
       

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot = criteria_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    c="Humidity",
    cmap="viridis",
    colorbar=True,
    title="City Humidity",
    height=500,
    width=800,
    size=city_data_df["Humidity"]*10,
    alpha=0.5,
    hover_cols=["City", "Country", "Humidity", "Hotel Name", "Country"]
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,_size,City,Country,Country)